In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cassandra/dataset/sample_submissions.csv
/kaggle/input/cassandra/dataset/train_data.csv
/kaggle/input/cassandra/dataset/test_data.csv


# Importing the libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
data=pd.read_csv('/kaggle/input/cassandra/dataset/train_data.csv')
data.head(2)

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,...,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
1,iNJcHQlPCm,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,...,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0


In [4]:
data.columns

Index(['UID', 'trip_creation_time', 'route_schedule_uuid', 'route_type',
       'trip_uuid', 'source_center', 'source_name', 'destination_center',
       'destination_name', 'od_start_time', 'od_end_time',
       'start_scan_to_end_scan', 'is_cutoff', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'osrm_time',
       'osrm_distance', 'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time'],
      dtype='object')

# 

# Checking the unique elements in route_type

In [5]:
data.route_type.unique()

array(['FTL', 'Carting'], dtype=object)

# checking the unique elements in source_name

In [6]:
data.source_name.unique()

array(['Jamshedpur_Central_I_3 (Jharkhand)',
       'Nashik_TgrniaRD_I (Maharashtra)', 'Raxaul_KairiyaT_D (Bihar)',
       ..., 'KharagpurBR_Central_DPP_1 (Bihar)',
       'PaliBirsighpr_CtyLgDPP_D (Madhya Pradesh)',
       'Maharajganj_PakriDPP_D (Uttar Pradesh)'], dtype=object)

In [7]:
test_data=pd.read_csv('/kaggle/input/cassandra/dataset/test_data.csv')

In [8]:
u=test_data
u.head(1)

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor
0,XMsT9clviB,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),2018-09-20 03:21:32.418600,...,27,2018-09-20 04:01:19.505586,27.637279,28.0,32.5395,1.428571,16.0,7.0,10.8152,2.285714


In [9]:
test_data['actual_time'] = 0.0 

# combining train and test data so that features will match after one hot encoding

In [10]:
combined_data = pd.concat([data, test_data], axis=0)

# dropping unnecessary columns

In [11]:
data.drop(['UID', 'route_schedule_uuid',
       'trip_uuid', 'source_center', 'destination_center'],axis='columns',inplace=True)
data.shape

(72158, 19)

In [12]:
combined_data.drop(['UID', 'route_schedule_uuid',
       'trip_uuid', 'source_center', 'destination_center'],axis='columns',inplace=True)
combined_data.shape

(144316, 19)

In [13]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144316 entries, 0 to 72157
Data columns (total 19 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   trip_creation_time              140801 non-null  object 
 1   route_type                      144316 non-null  object 
 2   source_name                     144316 non-null  object 
 3   destination_name                144316 non-null  object 
 4   od_start_time                   144316 non-null  object 
 5   od_end_time                     143596 non-null  object 
 6   start_scan_to_end_scan          144316 non-null  float64
 7   is_cutoff                       144316 non-null  bool   
 8   cutoff_factor                   144316 non-null  int64  
 9   cutoff_timestamp                143386 non-null  object 
 10  actual_distance_to_destination  144316 non-null  float64
 11  osrm_time                       129884 non-null  float64
 12  osrm_distance    

In [14]:
data.isnull().sum()

trip_creation_time                 3515
route_type                            0
source_name                           0
destination_name                      0
od_start_time                         0
od_end_time                         720
start_scan_to_end_scan                0
is_cutoff                             0
cutoff_factor                         0
cutoff_timestamp                    930
actual_distance_to_destination        0
osrm_time                         14432
osrm_distance                     14432
factor                             1644
segment_actual_time                   0
segment_osrm_time                     0
segment_osrm_distance                 0
segment_factor                     1159
actual_time                           0
dtype: int64

# checking missing,na,NaN etc

In [15]:
combined_data.isnull().sum()

trip_creation_time                 3515
route_type                            0
source_name                           0
destination_name                      0
od_start_time                         0
od_end_time                         720
start_scan_to_end_scan                0
is_cutoff                             0
cutoff_factor                         0
cutoff_timestamp                    930
actual_distance_to_destination        0
osrm_time                         14432
osrm_distance                     14432
factor                             1644
segment_actual_time                   0
segment_osrm_time                     0
segment_osrm_distance                 0
segment_factor                     1159
actual_time                           0
dtype: int64

In [16]:
data=data.dropna()
data.isnull().sum()

trip_creation_time                0
route_type                        0
source_name                       0
destination_name                  0
od_start_time                     0
od_end_time                       0
start_scan_to_end_scan            0
is_cutoff                         0
cutoff_factor                     0
cutoff_timestamp                  0
actual_distance_to_destination    0
osrm_time                         0
osrm_distance                     0
factor                            0
segment_actual_time               0
segment_osrm_time                 0
segment_osrm_distance             0
segment_factor                    0
actual_time                       0
dtype: int64

In [17]:
combined_data=combined_data.dropna()
combined_data.isnull().sum()

trip_creation_time                0
route_type                        0
source_name                       0
destination_name                  0
od_start_time                     0
od_end_time                       0
start_scan_to_end_scan            0
is_cutoff                         0
cutoff_factor                     0
cutoff_timestamp                  0
actual_distance_to_destination    0
osrm_time                         0
osrm_distance                     0
factor                            0
segment_actual_time               0
segment_osrm_time                 0
segment_osrm_distance             0
segment_factor                    0
actual_time                       0
dtype: int64

In [18]:
data.head(5)

,trip_creation_time,route_type,source_name,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,2018-09-15 02:23:18.656742,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
2,2018-09-23 22:15:46.635975,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0
3,2018-09-21 05:11:44.833176,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0
5,2018-10-01 18:06:41.031369,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),2018-10-01 18:06:41.031369,2018-10-03 20:49:39.290326,3145.532578,True,1474,2018-10-02 03:11:29,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,2489.0
6,2018-09-27 14:41:36.457278,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),2018-09-29 18:54:31.061435,2018-10-02 02:57:37.791139,3452.489276,True,1012,2018-09-30 21:02:29,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,1740.0


# Converting Columns into datetime

In [19]:
data.loc[:, 'trip_creation_time'] = pd.to_datetime(data['trip_creation_time'].copy())
data.loc[:, 'od_start_time'] = pd.to_datetime(data['od_start_time'].copy())
data.loc[:, 'od_end_time'] = pd.to_datetime(data['od_end_time'].copy())
data.loc[:, 'cutoff_timestamp'] = pd.to_datetime(data['cutoff_timestamp'].copy())



In [20]:
combined_data.loc[:, 'trip_creation_time'] = pd.to_datetime(combined_data['trip_creation_time'].copy())
combined_data.loc[:, 'od_start_time'] = pd.to_datetime(combined_data['od_start_time'].copy())
combined_data.loc[:, 'od_end_time'] = pd.to_datetime(combined_data['od_end_time'].copy())
combined_data.loc[:, 'cutoff_timestamp'] = pd.to_datetime(combined_data['cutoff_timestamp'].copy())



In [21]:
data.head()

,trip_creation_time,route_type,source_name,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,2018-09-15 02:23:18.656742,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
2,2018-09-23 22:15:46.635975,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0
3,2018-09-21 05:11:44.833176,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0
5,2018-10-01 18:06:41.031369,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),2018-10-01 18:06:41.031369,2018-10-03 20:49:39.290326,3145.532578,True,1474,2018-10-02 03:11:29,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,2489.0
6,2018-09-27 14:41:36.457278,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),2018-09-29 18:54:31.061435,2018-10-02 02:57:37.791139,3452.489276,True,1012,2018-09-30 21:02:29,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,1740.0


In [22]:
data.columns

Index(['trip_creation_time', 'route_type', 'source_name', 'destination_name',
       'od_start_time', 'od_end_time', 'start_scan_to_end_scan', 'is_cutoff',
       'cutoff_factor', 'cutoff_timestamp', 'actual_distance_to_destination',
       'osrm_time', 'osrm_distance', 'factor', 'segment_actual_time',
       'segment_osrm_time', 'segment_osrm_distance', 'segment_factor',
       'actual_time'],
      dtype='object')

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41305 entries, 0 to 72155
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              41305 non-null  datetime64[ns]
 1   route_type                      41305 non-null  object        
 2   source_name                     41305 non-null  object        
 3   destination_name                41305 non-null  object        
 4   od_start_time                   41305 non-null  datetime64[ns]
 5   od_end_time                     41305 non-null  datetime64[ns]
 6   start_scan_to_end_scan          41305 non-null  float64       
 7   is_cutoff                       41305 non-null  bool          
 8   cutoff_factor                   41305 non-null  int64         
 9   cutoff_timestamp                41305 non-null  datetime64[ns]
 10  actual_distance_to_destination  41305 non-null  float64       
 11  os

# doing features analysis and doing required

In [24]:
import pandas as pd

# Assuming 'column1' and 'column2' are the datetime columns in your DataFrame
data['time_difference_hours'] = (data['od_end_time'] - data['od_start_time']).dt.total_seconds() / 3600
data['cut_off_hours'] = (data['cutoff_timestamp'] - data['trip_creation_time']).dt.total_seconds() / 3600
data.drop(['od_end_time','od_start_time',
       'cutoff_timestamp','trip_creation_time'],axis='columns',inplace=True)
data.shape

(41305, 17)

In [25]:
combined_data['time_difference_hours'] = (combined_data['od_end_time'] - combined_data['od_start_time']).dt.total_seconds() / 3600
combined_data['cut_off_hours'] = (combined_data['cutoff_timestamp'] - combined_data['trip_creation_time']).dt.total_seconds() / 3600
combined_data.drop(['od_end_time','od_start_time',
       'cutoff_timestamp','trip_creation_time'],axis='columns',inplace=True)
combined_data.shape

(113463, 17)

In [26]:
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,time_difference_hours,cut_off_hours
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,4.171435,2.367873
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,7.306614,11.095379
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,9.147125,32.958935
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,2489.0,50.716183,9.079991
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,1740.0,56.051869,78.347929


In [27]:
data.shape

(41305, 17)

In [28]:
combined_data.shape

(113463, 17)

In [29]:
dummies = pd.get_dummies(data.is_cutoff)

dummies.head(3)


,False,True
0,0,1
2,0,1
3,0,1


In [30]:
dummies4 = pd.get_dummies(combined_data.is_cutoff)

dummies4.head(3)

,False,True
0,0,1
2,0,1
3,0,1


# Dropping Columns to avoid Dummy trap

In [31]:
data = pd.concat([data, dummies.drop(dummies.columns[0], axis='columns')], axis='columns')
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,time_difference_hours,cut_off_hours,True
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,4.171435,2.367873,1
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,7.306614,11.095379,1
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,9.147125,32.958935,1
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,2489.0,50.716183,9.079991,1
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,1740.0,56.051869,78.347929,1


In [32]:
combined_data = pd.concat([combined_data, dummies4.drop(dummies4.columns[0], axis='columns')], axis='columns')
combined_data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,time_difference_hours,cut_off_hours,True
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,4.171435,2.367873,1
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,7.306614,11.095379,1
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,9.147125,32.958935,1
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,2489.0,50.716183,9.079991,1
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,1740.0,56.051869,78.347929,1


In [33]:
dummies1 = pd.get_dummies(data.source_name)
dummies1.head(3)


,AMD_Memnagar (Gujarat),AMD_Rakhial (Gujarat),Abohar_DC (Punjab),Achrol_BgwriDPP_D (Rajasthan),Adoor_Town_D (Kerala),Agra_Central_D_3 (Uttar Pradesh),Agra_Idgah_L (Uttar Pradesh),Agra_Idgah_P (Uttar Pradesh),AhmedNagar_DC (Maharashtra),Ahmedabad_East_H_1 (Gujarat),...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_Central_D_2 (Telangana),Warangal_HunterRd_I (Telangana),Wardha_RamaNgr_D (Maharashtra),Weir_BsstdDPP_D (Rajasthan),YamunaNagar_DC (Haryana),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
dummies5 = pd.get_dummies(combined_data.source_name)
dummies5.head(3)

,AMD_Memnagar (Gujarat),AMD_Rakhial (Gujarat),Abohar_DC (Punjab),Achrol_BgwriDPP_D (Rajasthan),Addanki_Oilmilrd_D (Andhra Pradesh),Adoor_Town_D (Kerala),Agra_Central_D_3 (Uttar Pradesh),Agra_Idgah_L (Uttar Pradesh),Agra_Idgah_P (Uttar Pradesh),AhmedNagar_DC (Maharashtra),...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_Central_D_2 (Telangana),Warangal_HunterRd_I (Telangana),Wardha_RamaNgr_D (Maharashtra),Weir_BsstdDPP_D (Rajasthan),YamunaNagar_DC (Haryana),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
data = pd.concat([data, dummies1.drop(dummies1.columns[0], axis='columns')], axis='columns')
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_Central_D_2 (Telangana),Warangal_HunterRd_I (Telangana),Wardha_RamaNgr_D (Maharashtra),Weir_BsstdDPP_D (Rajasthan),YamunaNagar_DC (Haryana),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana)
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,0
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,0
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,0
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,0
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,0


In [36]:
combined_data = pd.concat([combined_data, dummies5.drop(dummies5.columns[0], axis='columns')], axis='columns')
combined_data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_Central_D_2 (Telangana),Warangal_HunterRd_I (Telangana),Wardha_RamaNgr_D (Maharashtra),Weir_BsstdDPP_D (Rajasthan),YamunaNagar_DC (Haryana),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana)
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,0
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,0
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,0
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,0
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,0


In [37]:
dummies2 = pd.get_dummies(data.destination_name)
dummies2.head(3)

,AMD_Memnagar (Gujarat),AMD_Satellite (Gujarat),Abohar_DC (Punjab),Achrol_BgwriDPP_D (Rajasthan),Addanki_Oilmilrd_D (Andhra Pradesh),Adoor_Town_D (Kerala),Agra (Uttar Pradesh),Agra_Idgah_L (Uttar Pradesh),Agra_Idgah_P (Uttar Pradesh),AhmedNagar_DC (Maharashtra),...,Wai_Central_DPP_3 (Maharashtra),Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
dummies6 = pd.get_dummies(combined_data.destination_name)
dummies6.head(3)

,AMD_Memnagar (Gujarat),AMD_Satellite (Gujarat),Abohar_DC (Punjab),Achrol_BgwriDPP_D (Rajasthan),Addanki_Oilmilrd_D (Andhra Pradesh),Adoor_Town_D (Kerala),Agra (Uttar Pradesh),Agra_Idgah_L (Uttar Pradesh),Agra_Idgah_P (Uttar Pradesh),AhmedNagar_DC (Maharashtra),...,Wai_Central_DPP_3 (Maharashtra),Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
data = pd.concat([data, dummies2.drop(dummies2.columns[0], axis='columns')], axis='columns')
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wai_Central_DPP_3 (Maharashtra),Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab)
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,0
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,0
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,0
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,0
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,0


In [40]:
combined_data = pd.concat([combined_data, dummies6.drop(dummies6.columns[0], axis='columns')], axis='columns')
combined_data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wai_Central_DPP_3 (Maharashtra),Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab)
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,0
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,0
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,0
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,0
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,0


In [41]:
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wai_Central_DPP_3 (Maharashtra),Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab)
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,0
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,0
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,0
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,0
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,0


In [42]:
dummies3 = pd.get_dummies(data.route_type)
dummies3.head(3)

,Carting,FTL
0,0,1
2,0,1
3,0,1


In [43]:
dummies7 = pd.get_dummies(combined_data.route_type)
dummies7.head(3)

,Carting,FTL
0,0,1
2,0,1
3,0,1


In [44]:
data = pd.concat([data, dummies3.drop(dummies3.columns[0], axis='columns')], axis='columns')
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,1
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,1
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,1
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,1
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,1


In [45]:
combined_data = pd.concat([combined_data, dummies7.drop(dummies7.columns[0], axis='columns')], axis='columns')
combined_data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,1
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,1
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,1
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,1
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,1


In [46]:
data.head()

,route_type,source_name,destination_name,start_scan_to_end_scan,is_cutoff,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,FTL,Jamshedpur_Central_I_3 (Jharkhand),Ranchi_Hub (Jharkhand),713.521495,True,44,158.807515,778.672716,1228.735753,2.782826,...,0,0,0,0,0,0,0,0,0,1
2,FTL,Raxaul_KairiyaT_D (Bihar),Muzaffrpur_Bbganj_I (Bihar),923.666146,True,22,134.486898,400.072069,1188.943158,4.415969,...,0,0,0,0,0,0,0,0,0,1
3,FTL,Muzaffrpur_Bbganj_I (Bihar),Purnia_Central_H_2 (Bihar),705.451567,True,176,262.866674,510.123878,999.421258,2.946647,...,0,0,0,0,0,0,0,0,0,1
5,FTL,Gurgaon_Bilaspur_HB (Haryana),Bangalore_Nelmngla_H (Karnataka),3145.532578,True,1474,1499.349234,2331.812110,2478.458117,2.513150,...,0,0,0,0,0,0,0,0,0,1
6,FTL,Kolkata_Dankuni_HB (West Bengal),Bhiwandi_Mankoli_HB (Maharashtra),3452.489276,True,1012,1091.565072,1421.126351,1749.869939,2.303103,...,0,0,0,0,0,0,0,0,0,1


In [47]:
df = data.drop(['route_type', 'is_cutoff', 'source_name', 'destination_name'], axis='columns')
df.head(2)

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,713.521495,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,...,0,0,0,0,0,0,0,0,0,1
2,923.666146,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,...,0,0,0,0,0,0,0,0,0,1


In [48]:
df1 = combined_data.drop(['route_type', 'is_cutoff', 'source_name', 'destination_name'], axis='columns')
df1.head(2)

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,713.521495,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,...,0,0,0,0,0,0,0,0,0,1
2,923.666146,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,...,0,0,0,0,0,0,0,0,0,1


In [49]:
num_train_samples = len(data)
train_data = df1[:num_train_samples]
testing_data = df1[num_train_samples:]


In [50]:
train_data.shape

(41305, 2957)

In [51]:
train_data['actual_time']

0         100.0
2          69.0
3         300.0
5        2489.0
6        1740.0
          ...  
72146    1723.0
72147      56.0
72148      31.0
72150      35.0
72155      74.0
Name: actual_time, Length: 41305, dtype: float64

In [52]:
testing_data.shape

(72158, 2957)

In [53]:
testing_data.drop(['actual_time'],axis='columns',inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
testing_data.head()

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,86.0,27,27.637279,28.0,32.5395,1.428571,16.0,7.0,10.8152,2.285714,...,0,0,0,0,0,0,0,0,0,0
1,86.0,39,39.386040,44.0,54.2181,1.545455,6.0,5.0,3.9153,1.200000,...,0,0,0,0,0,0,0,0,0,0
2,109.0,9,10.403038,11.0,12.1171,1.363636,15.0,11.0,12.1171,1.363636,...,0,0,0,0,0,0,0,0,0,0
3,109.0,36,38.939167,39.0,47.1900,1.948718,10.0,10.0,11.3648,1.000000,...,0,0,0,0,0,0,0,0,0,0
4,302.0,22,23.194334,24.0,26.8622,1.583333,38.0,24.0,26.8622,1.583333,...,0,0,0,0,0,0,0,0,0,1


In [55]:
X = train_data.drop(['actual_time'],axis='columns')
X.head(3)

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,713.521495,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,...,0,0,0,0,0,0,0,0,0,1
2,923.666146,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,...,0,0,0,0,0,0,0,0,0,1
3,705.451567,176,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,...,0,0,0,0,0,0,0,0,0,1


In [56]:
y=train_data.actual_time
y.shape

(41305,)

In [57]:
X.columns

Index([           'start_scan_to_end_scan',
                           'cutoff_factor',
          'actual_distance_to_destination',
                               'osrm_time',
                           'osrm_distance',
                                  'factor',
                     'segment_actual_time',
                       'segment_osrm_time',
                   'segment_osrm_distance',
                          'segment_factor',
       ...
       'Wanaparthy_VallaDPP_D (Telangana)',
           'Wankaner_JivanDPP_D (Gujarat)',
         'Warangal_HunterRd_I (Telangana)',
                'YamunaNagar_DC (Haryana)',
        'Yavatmal_JajuDPP_D (Maharashtra)',
         'Yellandu_Sudimala_D (Telangana)',
         'Yellareddy_JKRoad_D (Telangana)',
           'Zahirabad_Mohim_D (Telangana)',
                    'Zirakpur_DC (Punjab)',
                                     'FTL'],
      dtype='object', length=2956)

In [58]:
df1.head()

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,713.521495,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,...,0,0,0,0,0,0,0,0,0,1
2,923.666146,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,...,0,0,0,0,0,0,0,0,0,1
3,705.451567,176,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,...,0,0,0,0,0,0,0,0,0,1
5,3145.532578,1474,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,...,0,0,0,0,0,0,0,0,0,1
6,3452.489276,1012,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,...,0,0,0,0,0,0,0,0,0,1


# splitting into training and testing

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [60]:
X_train

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
6079,1287.035297,22,56.293386,541.137826,465.185343,2.796001,56.024690,20.210357,31.285877,2.124649,...,0,0,0,0,0,0,0,0,0,1
59303,662.136796,88,89.754906,590.069655,271.394976,3.704064,105.055816,34.255886,43.025252,5.640553,...,0,0,0,0,0,0,0,0,0,1
49994,1923.940905,132,264.421196,918.315994,905.270502,2.163372,81.238009,18.816086,23.270873,5.338413,...,0,0,0,0,0,0,0,0,0,1
13692,2028.162117,374,409.119918,822.529521,881.329646,2.721230,63.055532,17.099067,24.700121,3.633191,...,0,0,0,0,0,0,0,0,0,1
25879,2273.700183,330,340.357579,856.822606,717.249785,2.162364,32.279048,26.492515,33.877826,2.408198,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69983,555.674585,22,170.162701,677.367684,454.143509,3.789630,176.432826,63.416725,58.635320,3.275866,...,0,0,0,0,0,0,0,0,0,1
48988,3255.511962,1276,1446.452177,2035.251769,1892.899023,2.485590,43.235316,25.967601,38.909935,1.898441,...,0,0,0,0,0,0,0,0,0,1
51047,682.494494,66,122.762793,122.152820,926.663629,1.528979,54.460306,29.671471,30.887463,4.020838,...,0,0,0,0,0,0,0,0,0,1
69986,809.055834,81,366.531851,265.126108,1211.004025,2.396773,43.149374,17.106967,18.415260,2.441658,...,0,0,0,0,0,0,0,0,0,1


In [61]:
X_test

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
60620,672.100859,70,136.216415,270.814130,454.864054,3.591349,63.282385,36.589694,32.926345,3.317025,...,0,0,0,0,0,0,0,0,0,0
20777,621.833430,176,309.401195,614.426743,583.540436,2.041420,31.817289,19.329382,22.752292,2.713001,...,0,0,0,0,0,0,0,0,0,1
30340,295.108337,18,48.872833,1526.560513,866.965955,1.586912,33.624725,35.750935,34.146278,1.930522,...,0,0,0,0,0,0,0,0,0,0
61587,358.014179,27,53.324808,378.925570,426.913332,3.128032,14.466315,8.430212,15.201270,2.100598,...,0,0,0,0,0,0,0,0,0,0
30138,2961.062214,594,697.881736,850.990112,2347.323625,1.851530,38.167038,28.793196,35.757675,2.103441,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69123,506.347922,12,146.744656,447.308973,661.718477,2.842158,28.883884,4.720357,10.881884,4.967200,...,0,0,0,0,0,0,0,0,0,0
53682,2429.713803,1056,1076.172744,1225.909094,2008.445876,2.013962,16.655609,7.212310,21.556099,3.225204,...,0,0,0,0,0,0,0,0,0,1
2136,2721.942398,264,410.082753,757.693751,1017.593314,2.408389,32.237646,28.537585,28.107258,2.105299,...,0,0,0,0,0,0,0,0,0,1
39396,2228.535407,704,785.366069,800.406329,1354.380307,2.335714,45.994061,21.409288,36.441747,2.772146,...,0,0,0,0,0,0,0,0,0,1


# Applying models

In [62]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.9794451053801033

In [63]:
# from sklearn.model_selection import ShuffleSplit
# from sklearn.model_selection import cross_val_score

# cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# cross_val_score(LinearRegression(), X, y, cv=cv)

In [64]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
dt.score(X_test,y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.9831526830017466

In [65]:
y1_predicted=dt.predict(testing_data)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [66]:
X.head()

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,...,Wanaparthy_VallaDPP_D (Telangana),Wankaner_JivanDPP_D (Gujarat),Warangal_HunterRd_I (Telangana),YamunaNagar_DC (Haryana),Yavatmal_JajuDPP_D (Maharashtra),Yellandu_Sudimala_D (Telangana),Yellareddy_JKRoad_D (Telangana),Zahirabad_Mohim_D (Telangana),Zirakpur_DC (Punjab),FTL
0,713.521495,44,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,...,0,0,0,0,0,0,0,0,0,1
2,923.666146,22,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,...,0,0,0,0,0,0,0,0,0,1
3,705.451567,176,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,...,0,0,0,0,0,0,0,0,0,1
5,3145.532578,1474,1499.349234,2331.812110,2478.458117,2.513150,78.860571,35.429543,44.287435,2.661004,...,0,0,0,0,0,0,0,0,0,1
6,3452.489276,1012,1091.565072,1421.126351,1749.869939,2.303103,78.117604,18.785432,30.457445,5.088583,...,0,0,0,0,0,0,0,0,0,1


In [67]:
def predict_actual_time(start, end, start_scan_to_end_scan, cutoff_factor,
                        actual_distance_to_destination, osrm_time, osrm_distance,
                        factor, segment_actual_time, segment_osrm_time,
                        segment_osrm_distance, segment_factor):
    source_index = np.where(X.columns == start)[0][0]
    destination_index = np.where(X.columns == end)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = start_scan_to_end_scan
    x[1] = cutoff_factor
    x[2] = actual_distance_to_destination
    x[3] = osrm_time
    x[4] = osrm_distance
    x[5] = factor
    x[6] = segment_actual_time
    x[7] = segment_osrm_time
    x[8] = segment_osrm_distance
    x[9] = segment_factor
    if source_index >= 0:
        x[source_index] = 1
    if destination_index >= 0:
        x[destination_index] = 1

    return dt.predict([x])[0]


In [68]:
predict_actual_time('Jamshedpur_Central_I_3 (Jharkhand)','Ranchi_Hub (Jharkhand)',713,44,158,778,1228,2.7,77,20,28,3.2)

100.0

In [69]:
y_pred = dt.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['bool', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [70]:
from sklearn.ensemble import RandomForestClassifier  # for classification
# or
from sklearn.ensemble import RandomForestRegressor  # for regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # for classification, if applicable
# or
from sklearn.metrics import mean_squared_error  # for regression, if applicable


In [71]:
# model = RandomForestClassifier(n_estimators=100, random_state=42)  # for classification
# # or
# model = RandomForestRegressor(n_estimators=100, random_state=42)  # for regression
# model.fit(X_train, y_train)
# model.score(X_test,y_test)

In [75]:
# y2_predicted = model.predict(testing_data)


In [ ]:
from sklearn.svm import SVC




In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# from sklearn.model_selection import GridSearchCV

# from sklearn.linear_model import Lasso
# from sklearn.tree import DecisionTreeRegressor

# def find_best_model_using_gridsearchcv(X,y):
#     algos = {
#         'linear_regression' : {
#             'model': LinearRegression(),
#             'params': {
#                 'normalize': [True, False]
#             }
#         },
#         'lasso': {
#             'model': Lasso(),
#             'params': {
#                 'alpha': [1,2],
#                 'selection': ['random', 'cyclic']
#             }
#         },
#         'decision_tree': {
#             'model': DecisionTreeRegressor(),
#             'params': {
#                 'criterion' : ['mse','friedman_mse'],
#                 'splitter': ['best','random']
#             }
#            }
#     }
#     scores = []
#     cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
#     for algo_name, config in algos.items():
#         gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
#         gs.fit(X,y)
#         scores.append({
#             'model': algo_name,
#             'best_score': gs.best_score_,
#             'best_params': gs.best_params_
#         })

#     return pd.DataFrame(scores,columns=['model','best_score','best_params'])

# find_best_model_using_gridsearchcv(X,y)

In [ ]:
# output=pd.DataFrame(u["UID"],columns=["UID"])

In [ ]:
# output["actual_time"]=y2_predicted

In [ ]:
# output.head(2)

In [ ]:
# output.to_csv("output.csv",index=False)

In [ ]:
# X.to_csv("X.csv",index=False)

In [ ]:
# dr=pd.read_csv("X.csv")
# dr.head()